<a href="https://colab.research.google.com/github/adityasaicareer/Project_NUS/blob/main/main_file.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("andrewmvd/hard-hat-detection")

print("Path to dataset files:", path)

100%|██████████| 1.22G/1.22G [00:10<00:00, 129MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1


In [ ]:
path

'/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1'

In [ ]:
import glob

In [ ]:
glob.glob(path+'/annotations/*')

['/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers436.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers700.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers1474.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers3717.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers3776.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers1907.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers3334.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_workers2293.xml',
 '/root/.cache/kagglehub/datasets/andrewmvd/hard-hat-detection/versions/1/annotations/hard_hat_wor

In [ ]:
!pip install lxml

In [ ]:
from lxml import etree
temp=etree.parse(glob.glob(path+'/annotations/*')[0])

In [ ]:
newxml=etree.tostring(temp,pretty_print=True,encoding=str)

In [ ]:
print(newxml)

<annotation>
    <folder>images</folder>
    <filename>hard_hat_workers436.png</filename>
    <size>
        <width>416</width>
        <height>416</height>
        <depth>3</depth>
    </size>
    <segmented>0</segmented>
    <object>
        <name>helmet</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>158</xmin>
            <ymin>159</ymin>
            <xmax>187</xmax>
            <ymax>193</ymax>
        </bndbox>
    </object>
    <object>
        <name>helmet</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>
        <occluded>0</occluded>
        <difficult>0</difficult>
        <bndbox>
            <xmin>94</xmin>
            <ymin>139</ymin>
            <xmax>138</xmax>
            <ymax>198</ymax>
        </bndbox>
    </object>
    <object>
        <name>helmet</name>
        <pose>Unspecified</pose>
        <truncated>0</truncated>